## Setup and Data Loading

In [35]:
import geopandas as gpd
import pandas as pd
import folium
import branca.colormap as cm
from folium.plugins import Search, FastMarkerCluster
from branca.element import MacroElement
from jinja2 import Template
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from matplotlib.lines import Line2D
import contextily as ctx
from matplotlib.patches import Patch
import os

In [36]:
# Load Geospatial Data (Shapefile)
# Ensure the path matches your local folder structure
shp_path = "datasets/middle_layer/MSOA_2021_EW_BGC_V3.shp"
gdf_msoa = gpd.read_file(shp_path)

In [37]:
# Load Attribute Data
df_data = pd.read_csv("datasets/final_ds.csv")

In [38]:
# Load Site Facilities Data
df_sites = pd.read_csv("datasets/site_fac.csv")

In [39]:
# Merge based on the unique MSOA code
gdf_final = gdf_msoa.merge(df_data, left_on="MSOA21CD", right_on="MSOA code")

In [40]:
# CRS Transformation
# Folium requires WGS84 (EPSG:4326 - Lat/Lon), but UK data is usually in British National Grid (EPSG:27700)
gdf_final = gdf_final.to_crs(epsg=4326)

In [41]:
# Feature Engineering for Visualization
gdf_final['Wealth_PCT']   = (gdf_final['NS_1_2_prop'] * 100).round(1)
gdf_final['Poverty_PCT']  = (gdf_final['NS_8_prop'] * 100).round(1)
gdf_final['Active_PCT']   = (gdf_final['Active_All_adults'] * 100).round(1)
gdf_final['Inactive_PCT'] = (gdf_final['Inactive_All_adults'] * 100).round(1)
gdf_final['Gap_PCT']      = (gdf_final['Gender_Gap_Active'] * 100).round(1)
gdf_final['Minority_PCT'] = ((gdf_final['Asian_prop'] + gdf_final['Black_prop'] + gdf_final['Mixed_prop'] + gdf_final['Other_prop']) * 100).round(1)

gdf_final['Wealth_TXT']   = gdf_final['Wealth_PCT'].astype(str) + "%"
gdf_final['Poverty_TXT']  = gdf_final['Poverty_PCT'].astype(str) + "%"
gdf_final['Active_TXT']   = gdf_final['Active_PCT'].astype(str) + "%"
gdf_final['Inactive_TXT'] = gdf_final['Inactive_PCT'].astype(str) + "%"
gdf_final['Gap_TXT']      = gdf_final['Gap_PCT'].astype(str) + "%"
gdf_final['Minority_TXT'] = gdf_final['Minority_PCT'].astype(str) + "%"

# Handle NaNs
gdf_final['Facilities_Inside'] = gdf_final['Facilities_Inside'].fillna(0)
gdf_final['Diversity_Index_Inside'] = gdf_final['Diversity_Index_Inside'].fillna(0)

In [42]:
# Fixed Facility Type Mapping (5 and 8 are Outside Pitch, 4 and 17 are Tennis)
def recalculate_diversity(row):
    type_str = str(row['Type_List_Inside'])
    if pd.isna(type_str) or type_str == "" or type_str == "nan":
        return 0
    
    codes = [x.strip() for x in type_str.split(',')]
    unique_categories = set()
    
    for c in codes:
        if c in ['5', '8']:
            unique_categories.add('Outside Pitch')
        elif c in ['4', '17']: 
            unique_categories.add('Tennis')
        else:
            unique_categories.add(c)
            
    return len(unique_categories)

# Sovrascrivi la colonna usata poi dalla mappa
gdf_final['Diversity_Index_Inside'] = gdf_final.apply(recalculate_diversity, axis=1)

In [43]:
# "Mono-Football" Logic
# Identify areas without facilities (0), ONLY available facility is Football (Codes 5 or 8) (1), or Diverse facilities (2)
def classify_football(row):
    type_str = str(row['Type_List_Inside'])
    if pd.isna(type_str) or type_str == "" or type_str == "nan":
        return 0 # No Facilities
    
    codes = [x.strip() for x in type_str.split(',')]
    football_codes = ['5', '8']
    
    if len(codes) > 0 and all(c in football_codes for c in codes):
        return 1.5 # Mono-Football
    
    return 2.5 # Diverse

gdf_final['Football_Cat'] = gdf_final.apply(classify_football, axis=1)
gdf_final['Football_Label'] = gdf_final['Football_Cat'].map({0: "No Facilities", 1.5: "Only Football", 2.5: "Diverse"})

print(f"Data successfully loaded and processed. Total regions: {len(gdf_final)}")

Data successfully loaded and processed. Total regions: 6856


## Map Construction

In [44]:
# UTILITY CLASS
class BindColormap(MacroElement):
    """Binds a colormap to a given layer. Shows/Hides based on layer visibility."""
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            // 1. Hide the legend as soon as it is created
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
            
            // 2. Add listeners for when the user clicks
            {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)
print("Helper class 'BindColormap' defined successfully.")

Helper class 'BindColormap' defined successfully.


In [45]:
# LAYER CREATION FUNCTION
def add_custom_layer(map_obj, data, column, title, color_scheme, show=False, is_categorical=False, clip_quantile=None, legend_title=None):
    caption_text = legend_title if legend_title else title
    
    # 1. Colormap
    if is_categorical:
        cmap = cm.StepColormap(['#f0f0f0', '#e31a1c', '#08519c'], vmin=0, vmax=3, index=[0, 0.9, 1.9, 3], caption=caption_text)
    else:
        min_val = data[column].min()
        if clip_quantile:
            max_val = data[column].quantile(clip_quantile)
        else:
            max_val = data[column].max()
        cmap = cm.LinearColormap(colors=color_scheme, vmin=min_val, vmax=max_val, caption=caption_text)
    
    map_obj.add_child(cmap)
    
    # 2. GeoJson
    def style_fn(feature):
        val = feature['properties'][column]
        return {
            'fillColor': cmap(val) if pd.notnull(val) else 'gray',
            'color': 'black', 'weight': 0, 'fillOpacity': 0.8
        }

    layer = folium.GeoJson(
        data, style_function=style_fn, name=title,
        highlight_function=lambda x: {'weight': 2, 'color': 'black', 'fillOpacity': 0.9},
        show=show, smooth_factor=0.5
    ).add_to(map_obj)
    
    # 3. Bind Legend (hidden by default)
    map_obj.add_child(BindColormap(layer, cmap))
    
    # 4. If show=True, force legend to appear immediately
    if show:
        map_obj.get_root().html.add_child(folium.Element(f"""
        <script>
            document.addEventListener('DOMContentLoaded', function() {{
                {cmap.get_name()}.svg[0][0].style.display = 'block';
            }});
        </script>
        """))
        
    return layer

print("Layer creation function defined successfully.")

Layer creation function defined successfully.


In [46]:
# SITES DATA PREPARATION
df_sites = df_sites.dropna(subset=['Lat', 'Lon'])

facility_map = {
    "1": "Athletics", "2": "Gym", "4": "Tennis", "5": "Outside Pitch",
    "6": "Sports Hall", "7": "Swimming Pool", "8": "Outside Pitch", "9": "Golf",
    "10": "Ice Rinks", "11": "Ski Slope", "12": "Studios", "13": "Squash",
    "17": "Tennis", "20": "Cycling", "33": "Gymnastics"
}

def decode_facilities(code_str):
    if pd.isna(code_str): return "Unknown"
    codes = [c.strip() for c in str(code_str).split(',')]
    names = [facility_map.get(c, c) for c in codes]
    text = ", ".join(sorted(set(names)))
    return text.replace("'", "").replace('"', '').replace('\n', ' ').strip()

df_sites['Facility_Names'] = df_sites['Facility_Types_List'].apply(decode_facilities)
site_data = list(zip(df_sites['Lat'], df_sites['Lon'], df_sites['Facility_Names']))

callback_js = """
function (row) {
    var marker = L.marker(new L.LatLng(row[0], row[1]));
    var popupContent = "<div style='font-family:Arial; font-size:12px;'><b>Facilities:</b><br>" + row[2] + "</div>";
    marker.bindPopup(popupContent);
    return marker;
}
"""

print("Sites data prepared successfully.")

Sites data prepared successfully.


In [47]:
m = folium.Map(location=[52.5, -1.5], zoom_start=6, tiles="cartodbpositron")

# 1. Active (SHOW=TRUE)
add_custom_layer(m, gdf_final, 'Active_PCT', '1. Active Adults (%)', 
                 ['#d7191c', '#fdae61', '#ffffbf', '#a6d96a', '#1a9641'], show=True)

# 2. Inactive
add_custom_layer(m, gdf_final, 'Inactive_PCT', '2. Inactive Adults (%)', 
                 ['#1a9641', '#a6d96a', '#ffffbf', '#fdae61', '#d7191c'], show=False)

# 3. Gap
add_custom_layer(m, gdf_final, 'Gap_PCT', '3. Gender Play Gap (%)', 
                 ['#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026'], show=False)

# 4. Facilities
add_custom_layer(m, gdf_final, 'Facilities_Inside', '4. Number of Facilities', 
                 ['#fcfdbf', '#fc8961', '#b73779', '#51127c', '#000004'], 
                 show=False, clip_quantile=0.95)

# 5. Diversity
add_custom_layer(m, gdf_final, 'Diversity_Index_Inside', '5. Diversity Index', 
                 ['#fde725', '#5ec962', '#21918c', '#3b528b', '#440154'], show=False)

# 6. Facility Variety (Mono-Football vs Diverse)
add_custom_layer(m, gdf_final, 'Football_Cat', 
                 title='6. Facility Variety',
                 legend_title='White = None, Red = Only Football, Blue = 2+ Types',
                 color_scheme=None, show=False, is_categorical=True)

# 7. Sites Locations
FastMarkerCluster(
    data=site_data, callback=callback_js,
    name="7. Sports Sites Locations", show=False
).add_to(m)

tooltip = folium.features.GeoJsonTooltip(
    fields=['MSOA21NM', 'Active_TXT', 'Inactive_TXT', 'Gap_TXT', 'Facilities_Inside', 'Diversity_Index_Inside', 'Football_Label'],
    aliases=['Zone:', 'Active Adults:', 'Inactive Adults:', 'Gender Play Gap:', 'N. of Facilities:', 'Diversity Index:', 'Type:'],
    style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px; border: 1px solid gray;")
)

interaction_layer = folium.GeoJson(
    gdf_final,
    style_function=lambda x: {'fillColor': 'white', 'color': 'black', 'weight': 0, 'fillOpacity': 0},
    highlight_function=lambda x: {'weight': 2, 'color': 'black', 'fillOpacity': 0.1},
    tooltip=tooltip, name="Interaction Layer", control=False
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
m.keep_in_front(interaction_layer)
print("First Map (physical activity) created with all layers added successfully.")

First Map (physical activity) created with all layers added successfully.


In [48]:
outfile = "Physical_Activity_Map.html"
m.save(outfile)
print(f"Physical Activity & Facilities map saved as: {outfile}")

Physical Activity & Facilities map saved as: Physical_Activity_Map.html


In [49]:
m2 = folium.Map(location=[52.5, -1.5], zoom_start=6, tiles="cartodbpositron")

# 1. High Class (SHOW=TRUE)
add_custom_layer(m2, gdf_final, 'Wealth_PCT', '1. High Social Class (NS-SEC 1-2)', 
                 ['#eff3ff', '#bdd7e7', '#6baed6', '#3182bd', '#08519c'], show=True)

# 2. Low Class
add_custom_layer(m2, gdf_final, 'Poverty_PCT', '2. Low Social Class (NS-SEC 8)', 
                 ['#fff5eb', '#fdd0a2', '#fd8d3c', '#d94801', '#8c2d04'], show=False)

# 3. Minorities
add_custom_layer(m2, gdf_final, 'Minority_PCT', '3. Ethnic Minorities (%)', 
                 ['#fff7bc', '#fec44f', '#d95f0e', '#993404'], show=False)

tooltip2 = folium.features.GeoJsonTooltip(
    fields=['MSOA21NM', 'Wealth_TXT', 'Poverty_TXT', 'Minority_TXT'],
    aliases=['Zone:', 'High Class:', 'Low Class:', 'Minorities:'],
    style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px; border: 1px solid gray;")
)

interaction_layer2 = folium.GeoJson(
    gdf_final,
    style_function=lambda x: {'fillColor': 'white', 'color': 'black', 'weight': 0, 'fillOpacity': 0},
    highlight_function=lambda x: {'weight': 2, 'color': 'black', 'fillOpacity': 0.1},
    tooltip=tooltip2, name="Interaction Layer", control=False
).add_to(m2)

folium.LayerControl(collapsed=False).add_to(m2)
m2.keep_in_front(interaction_layer2)
print("Second Map (socio-economic) created with all layers added successfully.")

Second Map (socio-economic) created with all layers added successfully.


In [50]:
outfile2 = "Sociodemographic_Map.html"
m2.save(outfile2)
print(f"Socio-demographic map saved as: {outfile2}")

Socio-demographic map saved as: Sociodemographic_Map.html


## ISOCHRONE MAPPING

In [ ]:
# Isochrone parameters
trip_times = [10, 20, 30]  # in minutes
travel_speed = 4.5  # walking speed in km/h

# CHANGE NAME IF YOU WANT TO GENERETE ISOCHRONES FOR DIFFERENT MSOAS
place_names = [
    "Kensington, London, UK",
    "Newham, London, UK",
    "City of London, UK",
    "Croydon, London, UK",
    "Richmond upon Thames, UK"
]

In [52]:
# Facility type mapping
facility_map = {
    "1": "Athletics", "2": "Gym", "4": "Tennis", "5": "Outside Pitch",
    "6": "Sports Hall", "7": "Swimming Pool", "8": "Outside Pitch", "9": "Golf",
    "10": "Ice Rinks", "11": "Ski Slope", "12": "Studios", "13": "Squash",
    "17": "Tennis", "20": "Cycling", "33": "Gymnastics"
}

def decode_facilities(code_str):
    """
    Decode facility type codes into readable names
    
    Parameters:
    - code_str: comma-separated string of facility codes (e.g., "2, 4, 4, 7")
    
    Returns:
    - String with unique facility names (e.g., "Gym, Swimming Pool, Tennis")
    """
    if pd.isna(code_str):
        return "Unknown"
    
    # Split codes and clean whitespace
    codes = [c.strip() for c in str(code_str).split(',')]
    
    # Map codes to names
    names = [facility_map.get(c, c) for c in codes]
    
    # Get unique names and sort alphabetically
    unique_names = sorted(set(names))
    
    # Join with comma
    return ", ".join(unique_names)

# Apply to site_fac dataset
df_sites['Facility_Names'] = df_sites['Facility_Types_List'].apply(decode_facilities)

print("Facility names decoded successfully!")

Facility names decoded successfully!


In [53]:
def calculate_isochrones(center_lat, center_lon, trip_times, travel_speed=4.5):
    """
    Calculates isochrones for a given point
    
    Parameters:
    - center_lat, center_lon: coordinates of the starting point
    - trip_times: list of times in minutes (e.g., [10, 20, 30])
    - travel_speed: walking speed in km/h
    
    Returns:
    - GeoDataFrame containing the isochrones
    """
    
    # 1. Download the street network graph
    G = ox.graph_from_point((center_lat, center_lon), dist=3000, network_type='walk')
    G_proj = ox.project_graph(G)
    
    # 2. Find the nearest node to the center
    center_node = ox.distance.nearest_nodes(G, center_lon, center_lat)
    
    # 3. Add "time" attribute to edges
    meters_per_minute = travel_speed * 1000 / 60
    for orig, dest, key, data in G_proj.edges(data=True, keys=True):
        data["time"] = data["length"] / meters_per_minute
    
    # 4. Create isochrone polygons
    isochrone_polys = []
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G_proj, center_node, radius=trip_time, distance="time")
        node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
        bounding_poly = gpd.GeoSeries(node_points).union_all().convex_hull
        isochrone_polys.append(bounding_poly)
    
    # 5. Create GeoDataFrame
    crs_proj = ox.graph_to_gdfs(G_proj, nodes=True, edges=False).crs
    isochrones = gpd.GeoDataFrame(
        {'trip_time': sorted(trip_times, reverse=True), 
         'geometry': isochrone_polys},
        crs=crs_proj
    )
    
    # 6. Convert to WGS84 for Folium compatibility
    isochrones = isochrones.to_crs(epsg=4326)
    
    return isochrones

In [54]:
# Dictionary to store isochrones for each place
all_isochrones = {}

for place_name in place_names:
    print(f"Calculating isochrones for {place_name}...")
    
    # Geocode the location
    location = ox.geocode(place_name)
    
    # Calculate isochrones
    isochrones = calculate_isochrones(location[0], location[1], trip_times)
    
    # Save to dictionary
    all_isochrones[place_name] = {
        'isochrones': isochrones,
        'center': location
    }

Calculating isochrones for Kensington, London, UK...


Calculating isochrones for Newham, London, UK...
Calculating isochrones for City of London, UK...
Calculating isochrones for Croydon, London, UK...
Calculating isochrones for Richmond upon Thames, UK...


In [55]:
def count_sites_in_isochrones(isochrones_gdf, sites_gdf):
    """
    Counts how many sites and facilities are inside each isochrone.
    
    Parameters:
    - isochrones_gdf: GeoDataFrame containing isochrones
    - sites_gdf: GeoDataFrame containing sites (must have Lat, Lon, Total_Facilities columns)
    
    Returns:
    - DataFrame with counts
    """
    
    # Ensure sites_gdf has geometry
    if 'geometry' not in sites_gdf.columns:
        sites_gdf = sites_gdf.copy()
        sites_gdf['geometry'] = sites_gdf.apply(lambda row: Point(row['Lon'], row['Lat']), axis=1)
        sites_gdf = gpd.GeoDataFrame(sites_gdf, geometry='geometry', crs='EPSG:4326')
    
    results = []
    
    for idx, row in isochrones_gdf.iterrows():
        # Find sites inside this polygon
        sites_inside = sites_gdf[sites_gdf.within(row.geometry)]
        
        n_sites = len(sites_inside)
        n_facilities = sites_inside['Total_Facilities'].sum() if n_sites > 0 else 0
        
        results.append({
            'trip_time': row['trip_time'],
            'n_sites': n_sites,
            'n_facilities': int(n_facilities)
        })
    
    return pd.DataFrame(results)

In [56]:
# Count sites/facilities for each place's isochrones
for place_name in all_isochrones.keys():
    print(f"Counting sites and facilities for {place_name}")
    
    isochrones = all_isochrones[place_name]['isochrones']
    counts = count_sites_in_isochrones(isochrones, df_sites)
    
    all_isochrones[place_name]['counts'] = counts

Counting sites and facilities for Kensington, London, UK
Counting sites and facilities for Newham, London, UK
Counting sites and facilities for City of London, UK
Counting sites and facilities for Croydon, London, UK
Counting sites and facilities for Richmond upon Thames, UK


In [57]:
def create_isochrone_map(place_name, isochrones_gdf, center_coords, sites_gdf=None, counts_df=None):
    """
    Creating an interactive map with isochrones

    Parameters:
    - place_name: name of the place
    - isochrones_gdf: GeoDataFrame with isochrones
    - center_coords: tuple (lat, lon)
    - sites_gdf: optional, GeoDataFrame with the sites
    """
    
    # Initialize map
    m = folium.Map(location=center_coords, zoom_start=14, tiles='CartoDB Voyager')
    # Other options for tiles:
    # 'CartoDB positron'    - clean and light
    # 'OpenStreetMap'       - standard
    # 'Cartodb dark_matter' - dark theme
    # 'OpenTopoMap'         - topographic
    
    # Colors
    colors = ["#C6EEFC", "#8ED4F0", "#5BBEE1"]
    
    # Add isochrones
    for idx, row in isochrones_gdf.iterrows():
        folium.GeoJson(
            row.geometry,
            style_function=lambda x, color=colors[idx]: {
                'fillColor': color,
                'color': color,
                'weight': 2,
                'fillOpacity': 0.5
            },
            tooltip=f"{row['trip_time']} minuts walking"
        ).add_to(m)

    # Add sites markers if provided
    if sites_gdf is not None:
        # Ensure sites_gdf has geometry
        if 'geometry' not in sites_gdf.columns:
            sites_temp = sites_gdf.copy()
            sites_temp['geometry'] = sites_temp.apply(
                lambda row: Point(row['Lon'], row['Lat']), axis=1
            )
            sites_temp = gpd.GeoDataFrame(sites_temp, geometry='geometry', crs='EPSG:4326')
        else:
            sites_temp = sites_gdf
        
        # Get the largest isochrone (30 minutes) to filter sites
        largest_isochrone = isochrones_gdf.iloc[0].geometry  # first row is the largest (30 min)
        
        # Filter sites: only those within the largest isochrone
        sites_within = sites_temp[sites_temp.geometry.within(largest_isochrone)]
        
        # Prepare callback for markers with facilities popup
        site_data = []
        for idx, site in sites_within.iterrows():
            lat = site.geometry.y
            lon = site.geometry.x
            
            # Get facility names if available
            if 'Facility_Names' in site:
                facilities = site['Facility_Names']
            elif 'Total_Facilities' in site:
                facilities = f"{int(site['Total_Facilities'])} facilities"
            else:
                facilities = "Unknown"
            
            site_data.append([lat, lon, facilities])
        
        # JavaScript callback for custom markers
        callback_js = """
        function (row) {
            var marker = L.marker(new L.LatLng(row[0], row[1]), {
                icon: L.icon({
                    iconUrl: 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-2x-blue.png',
                    shadowUrl: 'https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/images/marker-shadow.png',
                    iconSize: [25, 41],
                    iconAnchor: [12, 41],
                    popupAnchor: [1, -34],
                    shadowSize: [41, 41]
                })
            });
            var popupContent = "<div style='font-family:Arial; font-size:12px;'><b>Facilities:</b><br>" + row[2] + "</div>";
            marker.bindPopup(popupContent);
            return marker;
        }
        """
        
        # Add FastMarkerCluster with blue markers
        FastMarkerCluster(
            data=site_data,
            callback=callback_js,
            name="Sports Sites"
        ).add_to(m)
        
    # Change center marker icon to house
    folium.Marker(
        center_coords,
        popup=place_name,
        icon=folium.Icon(color='red', icon='home', prefix='fa')  # house icon
    ).add_to(m)

    # Add legend with statistics
    if counts_df is not None:
        legend_html = '''
        <div style="position: fixed; 
                    top: 10px; right: 10px; width: 280px; height: auto; 
                    background-color: white; z-index:9999; font-size:14px;
                    border:2px solid grey; border-radius: 5px; padding: 10px">
        <h4 style="margin-top:0">{}</h4>
        <table style="width:100%; border-collapse: collapse;">
        <tr style="border-bottom: 1px solid #ddd;">
            <th style="text-align:left; padding:5px">Time (min)</th>
            <th style="text-align:right; padding:5px">Sites</th>
            <th style="text-align:right; padding:5px">Facilities</th>
        </tr>
        '''.format(place_name)
        
        for _, row in counts_df.sort_values(by='trip_time', ascending=True).iterrows():
            legend_html += f'''
            <tr style="border-bottom: 1px solid #eee;">
                <td style="padding:5px">{int(row["trip_time"])}</td>
                <td style="text-align:right; padding:5px">{int(row["n_sites"])}</td>
                <td style="text-align:right; padding:5px">{int(row["n_facilities"])}</td>
            </tr>
            '''
        
        legend_html += '</table></div>'
        m.get_root().html.add_child(folium.Element(legend_html))

    return m

In [58]:
# Create maps for each place and save as HTML
i = 0
for place_name, data in all_isochrones.items():
    print(f"Creating map for {place_name}")
    
    m = create_isochrone_map(
    place_name, 
    data['isochrones'], 
    data['center'],
    sites_gdf=df_sites,
    counts_df=data['counts']
    )
    
    # Save map
    clean_name = place_name.split(",")[0].replace(" ", "_")
    m.save(f"maps_output/isochrone_map_{clean_name}.html")
    
    # Show in the notebook only the first one
    # if i == 0:
    #     display(m)
    #     i = 1

Creating map for Kensington, London, UK
Creating map for Newham, London, UK
Creating map for City of London, UK
Creating map for Croydon, London, UK
Creating map for Richmond upon Thames, UK
